<a href="https://colab.research.google.com/github/kevin-rn/Grounding-LM/blob/main/abstractive_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cp -R ./drive/MyDrive/data/ ./

# Packages & Imports

In [3]:
%pip install -q pytorch-lightning
%pip install -q transformers
%pip install -q datasets
%pip install -q openai
# %pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.6/720.6 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from datasets import load_dataset, load_from_disk
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch.optim import AdamW
from tqdm.auto import tqdm
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

# Dataset

In [ ]:
# xsum_data = load_dataset("xsum")
# xsum_data.save_to_disk('data/xsum')

# cnn_data = load_dataset("cnn_dailymail", "3.0.0")
# cnn_data.save_to_disk('data/cnn_dailymail')

## tldr_data = load_dataset("webis/tldr-17")

In [5]:
data = load_from_disk('data/xsum')
# data = load_from_disk('data/cnn_dailymail')

df_train = pd.DataFrame(data=data['train'])
df_val = pd.DataFrame(data=data['validation'])
df_test = pd.DataFrame(data=data['test'])

df_train.columns = ['text', 'summary', 'id']
df_val.columns = ['text', 'summary', 'id']
df_test.columns = ['text', 'summary', 'id']

df_train.head()

,text,summary,id
0,"The full cost of damage in Newton Stewart, one...",Clean-up operations are continuing across the ...,35232142
1,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,40143035
2,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,35951548
3,"John Edward Bates, formerly of Spalding, Linco...",A former Lincolnshire Police officer carried o...,36266422
4,Patients and staff were evacuated from Cerahpa...,An armed man who locked himself into a room at...,38826984


In [6]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, text_max_len = 512, summary_max_len = 128):
        self.data = data
        self.tokenizer = tokenizer
        self.text_max_len = text_max_len
        self.summary_max_len = summary_max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):
        text = self.data.iloc[idx]['text']
        summary = self.data.iloc[idx]['summary']

        text_encoding = self.tokenizer(
            text,
            max_length=self.text_max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        summary_encoding = self.tokenizer(
            summary,
            max_length=self.summary_max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        labels = summary_encoding['input_ids']
        labels[labels == 0] = -100

        return {
            'text': text,
            'summary': summary,
            'input_ids': text_encoding['input_ids'],
            'attention_mask': text_encoding['attention_mask'],
            'labels': labels.flatten(),
            'labels_attention_mask': summary_encoding['attention_mask']
        }

In [7]:
class CustomDataModule(pl.LightningDataModule):
    def __init__(self, df_train, df_val, df_test, tokenizer, batch = 8, text_max_len = 512, summary_max_len = 128):
        super().__init__()
        self.df_train = df_train
        self.df_val = df_val
        self.df_test = df_test
        self.tokenizer = tokenizer
        self.batch = batch
        self.text_max_len = text_max_len
        self.summary_max_len = summary_max_len

    def setup(self, stage=None):
      self.train_dataset = CustomDataset(self.df_train, self.tokenizer, self.text_max_len, self.summary_max_len)
      self.val_dataset = CustomDataset(self.df_val, self.tokenizer, self.text_max_len, self.summary_max_len)
      self.test_dataset = CustomDataset(self.df_test, self.tokenizer, self.text_max_len, self.summary_max_len)

    def collate_fn(self, batch):
      texts = [item['text'] for item in batch]
      summaries = [item['summary'] for item in batch]
      text_input_ids = pad_sequence([item['input_ids'].flatten() for item in batch], batch_first=True)
      text_attention_masks = pad_sequence([item['attention_mask'].flatten() for item in batch], batch_first=True)
      labels = pad_sequence([item['labels'] for item in batch], batch_first=True)
      labels_attention_masks = pad_sequence([item['labels_attention_mask'].flatten() for item in batch], batch_first=True)

      return {
          'text': texts,
          'summary': summaries,
          'input_ids': text_input_ids,
          'attention_mask': text_attention_masks,
          'labels': labels,
          'labels_attention_mask': labels_attention_masks
      }

    def train_dataloader(self):
      return DataLoader(self.train_dataset, batch_size=self.batch, shuffle=True, num_workers=2, collate_fn=self.collate_fn)

    def val_dataloader(self):
      return DataLoader(self.val_dataset, batch_size=self.batch, shuffle=True, num_workers=2, collate_fn=self.collate_fn)

    def test_dataloader(self):
      return DataLoader(self.test_dataset, batch_size=self.batch, shuffle=True, num_workers=2, collate_fn=self.collate_fn)

### Model

In [ ]:
# class SummaryModel(pl.LightningModule):
  
#   def __init__(self):
#     super().__init__()
#     self.model = T5ForConditionalGeneration.from_pretrained('t5-base', return_dict=True)

#   def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):
    
#     output = self.model(input_ids=input_ids, 
#                         attention_mask=attention_mask, 
#                         decoder_attention_mask=decoder_attention_mask,
#                         labels=labels, 
#     )
#     return output
  
#   def training_step(self, batch, batch_idx):
#     input_ids = batch["input_ids"]
#     attention_mask = batch["attention_mask"]
#     decoder_attention_mask = batch["labels_attention_mask"]
#     labels = batch["labels"]

#     output = self.forward(input_ids, attention_mask, decoder_attention_mask, labels)
#     loss = output.loss
#     self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#     return loss

#   def validation_step(self, batch, batch_idx):
#     input_ids = batch["input_ids"]
#     attention_mask = batch["attention_mask"]
#     decoder_attention_mask = batch["labels_attention_mask"]
#     labels = batch["labels"]

#     output = self.forward(input_ids, attention_mask, decoder_attention_mask, labels)
#     loss = output.loss
#     self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#     return loss
  
#   def test_step(self, batch, batch_idx):
#     input_ids = batch["input_ids"]
#     attention_mask = batch["attention_mask"]
#     decoder_attention_mask = batch["labels_attention_mask"]
#     labels = batch["labels"]

#     output = self.forward(input_ids, attention_mask, decoder_attention_mask, labels)
#     loss = output.loss
#     self.log("test_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#     return loss
  
#   def configure_optimizers(self):
#     return AdamW(self.parameters(), lr=0.0001)

In [ ]:
# N_EPOCHS = 5
# BATCH_SIZE = 32

# tokenizer = T5Tokenizer.from_pretrained('t5-base')
# data_module = CustomDataModule(df_train, df_val, df_test, tokenizer, batch=BATCH_SIZE)
# model = SummaryModel()

# checkpoint_callback = ModelCheckpoint(dirpath="checkpoints", filename="best_checkpoints", save_top_k=1,
#                                          verbose=True, monitor="val_loss", mode="min")

# trainer = pl.Trainer(callbacks=checkpoint_callback, max_epochs=N_EPOCHS, accelerator="gpu", enable_progress_bar=True)

In [ ]:
# trainer.fit(model, data_module)

# trained_model = SummaryModel.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
# trained_model.freeze()

# Fine-Tuned models

### Pretrained T5 model

In [8]:
tokenizer = AutoTokenizer.from_pretrained("sysresearch101/t5-large-finetuned-xsum-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("sysresearch101/t5-large-finetuned-xsum-cnn")

### BART

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-xsum")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-xsum")

### GPT-2

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Ayham/xlmroberta_gpt2_summarization_xsum")
model = AutoModelForSeq2SeqLM.from_pretrained("Ayham/xlmroberta_gpt2_summarization_xsum")

# OPEN-AI Models


In [ ]:
import openai

openai.api_key = 'text_key'

### text-davinci-003

In [ ]:
def generate_summary(text):
  prompt = text + "\n\nTl;dr"
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    temperature=0.7,
    max_tokens=150,
    top_p=0.9,
    frequency_penalty=0.0,
    presence_penalty=1
  )
  summary_text = response['choices'][0]['text'].strip()
  return summary_text

In [ ]:
df_test['davinci3'] = df['text'].apply(generate_summary)

In [ ]:
df.to_csv('summary_output.csv', index=False)

### get-3.5-turbo

In [ ]:
def generate_summary(text):
  prompt = text + "\n\nTl;dr"
  response = openai.Completion.create(
    model="get-3.5-turbo",
    prompt=prompt,
    temperature=0.7,
    max_tokens=150,
    top_p=0.9,
    frequency_penalty=0.0,
    presence_penalty=1
  )
  summary_text = response['choices'][0]['text'].strip()
  return summary_text

In [ ]:
df_test['get-3.5-turbo'] = df['text'].apply(generate_summary)

In [ ]:
df.to_csv('summary_output.csv', index=False)

# Generate Summaries

In [12]:
data_module = CustomDataModule(df_train, df_val, df_test, tokenizer, batch=8)
data_module.setup()

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

summaries = []

with torch.no_grad():
    for batch in tqdm(data_module.test_dataloader()):
        input_ids = batch['input_ids'].to(device)
        summary_ids = model.generate(
          input_ids = input_ids,
          attention_mask=batch['attention_mask'].to(device),
          max_length=150, 
          num_beams=2,
          repetition_penalty=2.5,
          length_penalty=1.0,
          early_stopping=True
      )

        summary_text = [tokenizer.decode(sum_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for sum_id in summary_ids]
        summaries.extend(summary_text)

  0%|          | 0/1417 [00:00<?, ?it/s]

In [18]:
df_test['generated_summaries'] = summaries
df_test.to_csv("summary_output.csv", index=False)

In [20]:
%cp summary_output.csv ./drive/MyDrive/data/